# Modeling Continuous Data

In [ ]:
%matplotlib inline
import warnings
from IPython.display import display
warnings.filterwarnings('ignore')
# tag: remove-cell applied

## Quickstart

To run a continuous dataset:

In [ ]:
import pybmds

# create a continuous dataset
dataset = pybmds.ContinuousDataset(
    doses=[0, 25, 50, 75, 100],
    ns=[20, 20, 20, 20, 20],
    means=[6, 8, 13, 25, 30],
    stdevs=[4, 4.3, 3.8, 4.4, 3.7]
)

# create a session
session = pybmds.Session(dataset=dataset)
session.add_default_models()

# execute the session; recommend best-fitting model
session.execute_and_recommend()

# save excel report
df = session.to_df()
df.to_excel("output/report.xlsx")

# save to a word report
report = session.to_docx()
report.save("output/report.docx")

## Create a continuous dataset

To create a continuous dataset, you need a list of doses, mean responses, standard deviations, and the total number of subjects. As shown above, you use the ```ContinuousDataset()``` function and insert your data. You can specify the title of the dataset and the dose and response units. You can also create a plot of the dataset which will have the title that you name the dataset and the specified units. For example, here the dataset name is "Body Weight from ChemX Exposure".

In [ ]:
dataset = pybmds.ContinuousDataset(
    name="Body Weight from ChemX Exposure",
    dose_units="ppm",
    response_units="kg",
    doses=[0, 25, 50, 75, 100],
    ns=[20, 20, 20, 20, 20],
    means=[6, 8, 13, 25, 30],
    stdevs=[4, 4.3, 3.8, 4.4, 3.7]
)
dataset.plot()

## Fitting one model to dataset

If you want to fit only one model to your dataset, you can fit the specific model to the dataset and print the results such as the BMD, BMDL, BMDU, p-value, AIC, etc. Say you want to fit the Hill model, you can run the code below and print the results:

In [ ]:
from pybmds.models import continuous

model = continuous.Hill(dataset)
model.execute()
model.plot()

The summary output can also be displayed:


In [ ]:
print(model.text())

The individual models that you can fit are shown below. Note that the degrees of the Polynomial model can be mincreased to a maximum of the lesser of n-1 or 8 (as specified in the BMDS User Guide).

In [ ]:
continuous.Linear(dataset)
continuous.Polynomial(dataset, settings = {"degree": 2})
continuous.Polynomial(dataset, settings = {"degree": 3})
continuous.Power(dataset)
continuous.Hill(dataset)
continuous.ExponentialM3(dataset)
continuous.ExponentialM5(dataset)

## Changing initial model parameter settings

If you want to see a preview of the initial parameter settings, you can run:

In [ ]:
model = continuous.Hill(dataset)
print(model.settings.tbl())
print(model.priors_tbl())

You can also change the initial parameter settings shown above for any run of a single continuous model. For example, continuing with the Hill model example, you can set the power parameter `n` to be equal to 1. You do this by changing the initial, minimum, and maximum values of that parameter:

In [ ]:
model = continuous.Hill(dataset)
model.settings.priors.update('n', initial_value=1, min_value=1, max_value=1)
print(model.priors_tbl())

You can change the range and initial value for any parameter in the model by following the same steps above.

## Run all models and select the best fit

As shown in the Quickstart above, you can run all of the default continuous models and select the recommended model. To run all the default models, save the results, and save the plot of the fit of the recommended model with the data:

In [ ]:
# create a continuous dataset
dataset = pybmds.ContinuousDataset(
    doses=[0, 25, 50, 75, 100],
    ns=[20, 20, 20, 20, 20],
    means=[6, 8, 13, 25, 30],
    stdevs=[4, 4.3, 3.8, 4.4, 3.7]
)

session = pybmds.Session(dataset=dataset)
session.add_default_models()

# execute the session
session.execute()

# recommend a best-fitting model
session.recommend()

#print recommended model and plot recommended model with dataset
model_index = session.recommender.results.recommended_model_index
if model_index:
    model = session.models[model_index]
    model.plot()

You can also plot all models:

In [ ]:
session.plot()

In [ ]:
session.plot(colorize=False)

## Run subset of models and select best fit

You can select a set of models and find the best fit, rather than using all of the default continuous models. For example, to model average on the Hill, Linear, and Exponential 3, you can:

In [ ]:
session1 = pybmds.Session(dataset=dataset)
session1.add_model(pybmds.Models.Linear)
session1.add_model(pybmds.Models.ExponentialM3)
session1.add_model(pybmds.Models.Hill)

session1.execute()
session1.recommend()

model_index = session1.recommender.results.recommended_model_index
if model_index:
    model = session1.models[model_index]
    model.plot()

## Changing the input settings

The default settings when running a continuous analysis are a BMR of 1 standard deviation from the BMD, a 95% confidence interval, and a normally distributed and constant variance between dose groups. If you fit just one model to your dataset, you can change these settings by:

In [ ]:
model = continuous.Hill(dataset)
print(model.settings.tbl())
print(model.priors_tbl())

You can change individual model settings:


In [ ]:
model = continuous.Hill(
    dataset,
    settings={
        "bmr": 0.1,
        "bmr_type": pybmds.ContinuousRiskType.RelativeDeviation,
        "disttype": pybmds.ContinuousDistType.normal_ncv
    }
)
model.settings.priors.update('k', max_value=3)
print(model.settings.tbl())
print(model.priors_tbl())

### Changing all model settings for a Session


You can change model settings as well:

In [ ]:
session = pybmds.Session(dataset=dataset)
session.add_default_models(
    global_settings = {
        "disttype": pybmds.ContinuousDistType.normal_ncv,
        "bmr_type": pybmds.ContinuousRiskType.AbsoluteDeviation,
        "bmr": 2,
        "alpha": 0.1
    }
)

Here, we changed the settings to assume normally distributed and non-constant variance between dose groups and a 90% confidence interval. We also changed the BMR type to be absolute deviation with a value of 2. You can use the default BMR for the given BMR type or specify a different value. You can also change the`disttype` to be `DistType.log_normal`. If a log-normal distribution is selected, only the Hill and Exponential models will be run.